In [ ]:
import numpy as np
import pandas as pd
from keras.applications.xception import Xception
from keras.preprocessing.image import load_img, img_to_array
#from keras.utils import load_img,img_to_array
from keras.layers import Dense, Input
from keras.models import Model
from keras.utils import to_categorical
from sklearn.metrics import classification_report

Load data

In [ ]:
train_data = pd.read_csv('task_humanitarian_text_img_train.csv')
test_data = pd.read_csv('task_humanitarian_text_img_test.csv')
val_data = pd.read_csv('task_humanitarian_text_img_dev.csv')

In [ ]:
X1_image=[]
Y1_image=[]
X2_image=[]
Y2_image=[]
X3_image=[]
Y3_image=[]

In [ ]:
for i in range(0,13608):
    if train_data['label_text'][i]==train_data['label_image'][i]:
        X1_image.append(train_data['image'][i])
        Y1_image.append(train_data['label_image'][i])

In [ ]:
for i in range(0,2237):
    if test_data['label_text'][i]==test_data['label_image'][i]:
        X2_image.append(test_data['image'][i])
        Y2_image.append(test_data['label_image'][i])

In [ ]:
for i in range(0,2237):
    if val_data['label_text'][i]==val_data['label_image'][i]:
        X3_image.append(val_data['image'][i])
        Y3_image.append(val_data['label_image'][i])

Load image data

In [ ]:
img_width, img_height = 100, 100
num_classes = 8
img_train = []
img_test = []
img_val = []

In [ ]:
for filename in X1_image:
    img = load_img( filename, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_train.append(img_array)

In [ ]:
for filename in X2_image:
    img = load_img( filename, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_test.append(img_array)

In [ ]:
for filename in X3_image:
    img = load_img( filename, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_val.append(img_array)

In [ ]:
img_train = np.array(img_train)
img_test = np.array(img_test)
img_val = np.array(img_val)

Preprocess data

In [ ]:
img_train = img_train / 255.0
img_test = img_test / 255.0
img_val = img_val / 255.0

In [ ]:
tempTrain=[]
tempTest=[]
tempVal=[]

for i in range(0, len(Y1_image)):
    if train_data['label'][i] == 'affected_individuals':
        tempTrain.append(0)
    elif train_data['label'][i] == 'infrastructure_and_utility_damage':
        tempTrain.append(1)
    elif train_data['label'][i] == 'injured_or_dead_people':
        tempTrain.append(2)
    elif train_data['label'][i] == 'missing_or_found_people':
        tempTrain.append(3)
    elif train_data['label'][i] == 'not_humanitarian':
        tempTrain.append(4)
    elif train_data['label'][i] == 'other_relevant_information':
        tempTrain.append(5)
    elif train_data['label'][i] == 'rescue_volunteering_or_donation_effort':
        tempTrain.append(6)
    elif train_data['label'][i] == 'vehicle_damage':
        tempTrain.append(7)

for i in range(0, len(Y2_image)):
    if test_data['label'][i] == 'affected_individuals':
        tempTest.append(0)
    elif test_data['label'][i] == 'infrastructure_and_utility_damage':
        tempTest.append(1)
    elif test_data['label'][i] == 'injured_or_dead_people':
        tempTest.append(2)
    elif test_data['label'][i] == 'missing_or_found_people':
        tempTest.append(3)
    elif test_data['label'][i] == 'not_humanitarian':
        tempTest.append(4)
    elif test_data['label'][i] == 'other_relevant_information':
        tempTest.append(5)
    elif test_data['label'][i] == 'rescue_volunteering_or_donation_effort':
        tempTest.append(6)
    elif test_data['label'][i] == 'vehicle_damage':
        tempTest.append(7)

for i in range(0, len(Y3_image)):
    if val_data['label'][i] == 'affected_individuals':
        tempVal.append(0)
    elif val_data['label'][i] == 'infrastructure_and_utility_damage':
        tempVal.append(1)
    elif val_data['label'][i] == 'injured_or_dead_people':
        tempVal.append(2)
    elif val_data['label'][i] == 'missing_or_found_people':
        tempVal.append(3)
    elif val_data['label'][i] == 'not_humanitarian':
        tempVal.append(4)
    elif val_data['label'][i] == 'other_relevant_information':
        tempVal.append(5)
    elif val_data['label'][i] == 'rescue_volunteering_or_donation_effort':
        tempVal.append(6)
    elif val_data['label'][i] == 'vehicle_damage':
        tempVal.append(7)
        
labels_train = tempTrain
labels_test = tempTest
labels_val = tempVal
Y_train = to_categorical(labels_train, num_classes=num_classes)
Y_test = to_categorical(labels_test, num_classes=num_classes)
Y_val = to_categorical(labels_val, num_classes=num_classes)

Define Xception model

In [ ]:
xception_input = Input(shape=(img_width, img_height, 3))
xception_model = Xception(weights='imagenet', include_top=False, input_tensor=xception_input, pooling='max')
xception_output = Dense(num_classes, activation='softmax')(xception_model.output)
xception_model = Model(inputs=xception_input, outputs=xception_output)

Compile the model

In [ ]:
xception_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Train the model

In [ ]:
xception_model.fit(img_train, Y_train, batch_size=40, epochs=10, validation_data=(img_val, Y_val))

Evaluate the model

In [ ]:
_, accuracy = xception_model.evaluate(img_test, Y_test)
print('Accuracy: %.2f' % (accuracy*100))

Make predictions on the test dataset

In [ ]:
y_pred = xception_model.predict(img_test)
y_pred_classes = np.argmax(y_pred, axis=1)

Convert labels from one-hot encoding to integers

In [ ]:
y_test_classes = np.argmax(Y_test, axis=1)

Generate classification report

In [ ]:
report = classification_report(y_test_classes, y_pred_classes)
print(report)